# ARCH模型

## 原理讲解

### 产生背景

通常认为，横截面数据容易存在异方差，而时间序列数据常存在自相关。然而，Engle（1982）指出，时间序列数据也常存在一种特殊的异方差，即“自回归条件异方差”（Autoregressive Conditional Heteroskedasticity，简记 ARCH）Bollerslev（1986）对 ARCH 进行了推广，称为“ Generalized ARCH“，简记 GARCH
考察美国道琼斯股指在1953—1990年期间日收益率的波动，参见下图

<div align=center><img src="https://gitee.com/lei940324/picture/raw/master/img/2020/0516/231510.png" width="499" ></div>

从图可以看出，股指日收益率在某一段时间内剧烈波动，而在另一段时间内风平浪静。从理论上，这可以抽象为，当本期或过去若干期的波动（方差）较大时，未来几期的波动（方差）很可能也较大；反之亦然。换言之，方差大的观测值似乎集聚在一起，而方差小的观测值似乎也集聚在一起。这被称为“波动性集聚“（ volatility clustering）或“扎堆“。

### ARCH 模型的性质

考虑一般的线性回归模型:

$$y_{t}=x_{t}^{\prime} \boldsymbol{\beta}+\varepsilon_{t}\tag{1}$$

记扰动项 $\varepsilon_{t}$ 的条件方差为$\sigma_{t}^{2} \equiv \operatorname{Var}\left(\varepsilon_{t} | \varepsilon_{t-1}, \cdots\right)$，其中 $\sigma_{t}^{2}$ 的下标 $t$ 表示条件方差可以随时间而变。受到波动性集聚现象的启发，假设 $\sigma_{t}^{2}$ 取决于上一期扰动项之平方：

$$\sigma_{t}^{2}=\alpha_{0}+\alpha_{1} \varepsilon_{t-1}^{2}\tag{2}$$

这就是“ ARCH(1)扰动项”。更一般地，假设 $\sigma_{t}^{2}$ 依赖于前 $p$ 期扰动项之平方:

$$\sigma_{t}^{2}=\alpha_{0}+\alpha_{1} \varepsilon_{t-1}^{2}+\cdots+\alpha_{p} \varepsilon_{t-p}^{2}\tag{3}$$

就是“ ARCH(p)扰动项”。不失一般性，以 ARCH(1)扰动项为例，来考察 ARCH 扰动项的性质。假设扰动项 $\varepsilon_{t}^{2}$ 的生成过程为:

$$\varepsilon_{t}=v_{t} \sqrt{\alpha_{0}+\alpha_{1} \varepsilon_{t-1}^{2}}\tag{4}$$

其中，$v_{t}$ 为白噪声，并将其方差标准化为 1，即 $\operatorname{Var}\left(v_{t}\right)=\mathrm{E}\left(v_{t}^{2}\right)=1$。假定 $v_{t}$ 与$\epsilon _ { t - 1 }$相互独立，而且$\alpha _ { 0 } > 0$，$0 < \alpha _ { 1 } < 1$（为了保证 $\sigma_{t}^{2}$ 为正，且 {$\varepsilon_{t}^{2}$} 为平稳过程）。序列 {$\varepsilon_{t}^{2}$} 具有怎样的性质呢？下面我们来考察其条件期望、无条件期望、条件方差、无条件方差及序列相关。

由于 $v _ { t }$ 与 $\epsilon _ { t - 1 }$ 相互独立，$\epsilon_t$ 的条件期望为

$$\begin{aligned}
\mathrm{E}\left(\varepsilon_{t} | \varepsilon_{t-1}\right)=& \mathrm{E}\left\{v_{t} \sqrt{\alpha_{0}+\alpha_{1} \varepsilon_{t-1}^{2}} | \varepsilon_{t-1}\right\} \\
=& \underbrace{\mathrm{E}\left(v_{t}\right)}_{=0} \cdot \mathrm{E}\left\{\sqrt{\alpha_{0}+\alpha_{1} \varepsilon_{t-1}^{2}} | \varepsilon_{t-1}\right\}=0
\end{aligned}\tag{5}$$
